In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import torch

# Importing libraries
import copy
import os

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset
from typing import Tuple, List
from torch.optim import Optimizer

from nltk.corpus import words, wordnet
import nltk

import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

BASE_PATH = 'drive/MyDrive/NLP_project/'
# BASE_PATH = './'
if not os.path.exists(BASE_PATH):
    raise ValueError('path does not exist')

In [4]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


# Load dataset

In [6]:
df_train = pd.read_csv(BASE_PATH + 'datasets/EmoTrain.csv')
df_test = pd.read_csv(BASE_PATH + 'datasets/EmoVal.csv')

In [7]:
df_train = df_train.drop(axis=1, columns=['Unnamed: 0'])
df_test = df_test.drop(axis=1, columns=['Unnamed: 0'])
df_train

,text,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,Is there some scripture you could quote me? I'...,1,0,0,0,0,1,0,0,0,0
1,Good. Now we just need people to dislike commi...,1,0,0,0,0,0,0,0,0,0
2,This was driving me NUTS!,0,1,0,0,0,0,0,0,0,0
3,Thank you for your advice!,0,0,0,0,0,0,0,0,0,1
4,Some do. Some don't. Blanket generalizations a...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
190097,They needed to insert the phrase “over mainten...,0,0,0,0,1,0,0,0,0,0
190098,Back in the seventies and eighties we all did ...,1,0,0,0,0,0,0,0,0,0
190099,"6lbs is a lap dog, if someone shoots that caus...",0,0,0,0,0,0,0,1,0,0
190100,This gets much worse on the 2nd loop.,0,0,0,0,1,0,0,0,0,0


In [8]:
# step = 5_000
df_train = df_train[:190_000]
df_test = df_test[:190_000]
# df_train = df_train[:100_000]
# df_test = df_test[:100_000]

# Pre processing

## Description of data

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190000 entries, 0 to 189999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   text         190000 non-null  object
 1   praise       190000 non-null  int64 
 2   amusement    190000 non-null  int64 
 3   anger        190000 non-null  int64 
 4   disapproval  190000 non-null  int64 
 5   confusion    190000 non-null  int64 
 6   interest     190000 non-null  int64 
 7   sadness      190000 non-null  int64 
 8   fear         190000 non-null  int64 
 9   joy          190000 non-null  int64 
 10  love         190000 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 15.9+ MB


In [10]:
print(df_train.isnull().sum())

text           0
praise         0
amusement      0
anger          0
disapproval    0
confusion      0
interest       0
sadness        0
fear           0
joy            0
love           0
dtype: int64


In [11]:
def split_features_labels(df) -> Tuple[pd.Series, pd.DataFrame]:
    x = df['text']
    y = df.drop(axis=1, columns=['text'])
    return x, y


x_train, y_train = split_features_labels(df_train)
x_test, y_test = split_features_labels(df_test)

In [12]:
x_train

,text
0,Is there some scripture you could quote me? I'...
1,Good. Now we just need people to dislike commi...
2,This was driving me NUTS!
3,Thank you for your advice!
4,Some do. Some don't. Blanket generalizations a...
...,...
189995,"I'd be so upset about this, I hope you got to ..."
189996,9-3 in first straw vote? i was not aware of that.
189997,Yeah I know day of the dead is real thing but ...
189998,Wrong. You tried to use a WW2 example and it b...


In [13]:
y_train

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
189995,0,0,0,0,0,0,1,0,0,0
189996,0,0,0,0,1,0,0,0,0,0
189997,0,0,0,0,0,0,0,0,1,0
189998,0,0,0,1,0,0,0,0,0,0


## Data cleaning

Lower case all the words

In [14]:
def lower_case(x):
    x = x.str.lower()
    return x

x_train = lower_case(x_train)
x_test = lower_case(x_test)

In [15]:
x_train

,text
0,is there some scripture you could quote me? i'...
1,good. now we just need people to dislike commi...
2,this was driving me nuts!
3,thank you for your advice!
4,some do. some don't. blanket generalizations a...
...,...
189995,"i'd be so upset about this, i hope you got to ..."
189996,9-3 in first straw vote? i was not aware of that.
189997,yeah i know day of the dead is real thing but ...
189998,wrong. you tried to use a ww2 example and it b...


Convert to tokens

In [16]:
x_train = x_train.to_list()
x_test = x_test.to_list()

In [17]:
print(len(x_train))
print(x_train[:5])

190000
["is there some scripture you could quote me? i'd like to read up on it just to be sure for myself", 'good. now we just need people to dislike commies more than they do now', 'this was driving me nuts!', 'thank you for your advice!', "some do. some don't. blanket generalizations are almost always false and unhelpful."]


In [18]:
def list_of_words(x):
    x = list(map(lambda i: i.split(), x))
    return x

x_train = list_of_words(x_train)
x_test = list_of_words(x_test)

Then we have **list** of **list** of **tokens**

In [19]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me?',
  "i'd",
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself'],
 ['good.',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts!'],
 ['thank', 'you', 'for', 'your', 'advice!'],
 ['some',
  'do.',
  'some',
  "don't.",
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful.'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size.'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to.',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article.',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  "i'm",
  'feeling',
  'pumped',
  'about',
  'start

What kind of **characters** are there in the dataset?

In [20]:
characters = {'isalnum': 0}
for text in x_train:
    for word in text:
        for c in word:
            if c.isalnum():
                characters['isalnum'] += 1
            elif c in characters:
                characters[c] += 1
            else:
                characters[c] = 0

In [21]:
print(len(characters))
keys = list(characters.keys())
keys.sort()
print(keys)
characters

464
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'isalnum', '{', '|', '}', '~', '¡', '£', '¦', '©', '«', '¬', '¯', '°', '´', '·', '»', '̕', '̖', '̗', '̘', '̙', '̜', '̝', '̞', '̟', '̠', '̢', '̣', '̤', '̥', '̦', '̧', '̨', '̪', '̫', '̬', '̭', '̮', '̯', '̰', '̱', '̲', '̳', '̶', '̷', '̹', '̺', '̻', '̼', 'ͅ', '͈', '͍', '͎', '͏', '͓', '͔', '͕', '͖', '͘', '͙', '͚', '͜', '͝', '͞', '͟', '͠', '͡', '͢', '׳', '\u200d', '–', '—', '‘', '’', '“', '”', '„', '•', '…', '\u202a', '\u202c', '‽', '€', '™', '√', '≈', '≠', '▀', '▫', '☁', '☂', '☆', '☕', '☝', '☠', '☹', '☺', '♀', '♂', '♡', '♥', '♪', '♫', '♭', '♾', '♿', '⚔', '⚡', '⚰', '⛏', '⛑', '✊', '✋', '✌', '✔', '✨', '❄', '❣', '❤', '⠀', '⠁', '⠃', '⠄', '⠇', '⠈', '⠉', '⠊', '⠋', '⠏', '⠓', '⠘', '⠙', '⠚', '⠛', '⠞', '⠟', '⠢', '⠦', '⠫', '⠳', '⠴', '⠶', '⠸', '⠹', '⠻', '⠾', '⠿', '⡀', '⡄', '⡆', '⡇', '⡏', '⡜', '⡟', '⡶', '⡷', '⡼', '⡾', '⡿', '⢀', '⢠', '⢣', '⢤', '⢧', '⢰', '⢸', '⢹'

{'isalnum': 10313260,
 '?': 23237,
 "'": 65607,
 '.': 206936,
 '!': 37647,
 '’': 29710,
 '[': 34037,
 ']': 34022,
 ',': 62943,
 '"': 10798,
 ':': 6842,
 ')': 5247,
 '-': 8628,
 '💩': 2,
 '/': 5537,
 '*': 8644,
 '(': 3892,
 '^': 1507,
 '>': 2990,
 '~': 999,
 '”': 2182,
 '😒': 18,
 '£': 50,
 '“': 2204,
 '🙄': 92,
 '🍻': 7,
 '—': 112,
 '😂': 1141,
 '=': 421,
 '$': 760,
 '😍': 107,
 '😪': 14,
 ';': 880,
 '&': 356,
 '😭': 226,
 '❤': 355,
 '️': 722,
 '%': 929,
 '͡': 49,
 '͜': 32,
 '🅱': 33,
 '😔': 47,
 '+': 516,
 '🤕': 7,
 '@': 122,
 '🤣': 319,
 '#': 755,
 '🙌': 46,
 '😇': 10,
 '🎶': 50,
 '💃': 22,
 '<': 483,
 '\u200d': 223,
 '♀': 143,
 '☺': 57,
 '😢': 79,
 '😦': 1,
 '😜': 12,
 '♡': 3,
 '_': 608,
 '‘': 340,
 '💪': 26,
 '🤔': 154,
 '😣': 12,
 '😊': 100,
 '🐢': 2,
 '😄': 26,
 '😐': 32,
 '😤': 273,
 '😞': 14,
 '¦': 2,
 '¯': 83,
 '😱': 24,
 '👍': 129,
 '🍑': 2,
 '😹': 2,
 '👌': 84,
 '🏾': 11,
 '😕': 19,
 '♂': 76,
 '🏻': 128,
 '–': 13,
 '💕': 37,
 '❣': 10,
 '😎': 86,
 '😡': 48,
 '🤬': 11,
 '😴': 5,
 '€': 32,
 '👏': 173,
 '™': 30,
 '🤢': 5

- Remove non-semantic characters
- Convert `?` and `!` and `emojis` to tokens

In [22]:
def split_special_chars(word: str):
    if word.isalnum():
        return word, []
    specials = list(filter(lambda c: c == '!' or c == '?', word))
    return ''.join(filter(lambda c: c.isalnum(), word)), specials

def clean_tokens(x):
    for text in tqdm(x):
        specials = []
        for i, word in enumerate(text):
            text[i], special_word = split_special_chars(word)
            specials.extend(special_word)
        text.extend(specials)

clean_tokens(x_train)
clean_tokens(x_test)

  0%|          | 0/190000 [00:00<?, ?it/s]

  0%|          | 0/10562 [00:00<?, ?it/s]

In [23]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me',
  'id',
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself',
  '?'],
 ['good',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts', '!'],
 ['thank', 'you', 'for', 'your', 'advice', '!'],
 ['some',
  'do',
  'some',
  'dont',
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  'im',
  'feeling',
  'pumped',
  'about',
  's

Remove stop-words

In [24]:
print(len(ENGLISH_STOP_WORDS))
print(ENGLISH_STOP_WORDS)

318
frozenset({'take', 'must', 'throughout', 'upon', 'until', 'anywhere', 'themselves', 'thereafter', 'again', 'thus', 'seemed', 'some', 'due', 'ten', 'me', 'along', 'that', 'perhaps', 'an', 'too', 'if', 'seems', 'herself', 'seem', 'un', 'go', 'become', 'everything', 'still', 'describe', 'sixty', 'fifty', 'how', 'as', 'alone', 'eight', 'under', 'even', 'this', 'our', 'they', 'couldnt', 'to', 'out', 'etc', 'find', 'which', 'him', 'thereby', 'besides', 'hereafter', 'am', 'his', 'yet', 'below', 'together', 'at', 'whose', 'empty', 'same', 'through', 'around', 'only', 'should', 'cant', 'part', 'always', 'else', 'these', 'against', 'mostly', 'already', 'its', 'amongst', 'see', 'into', 'who', 'co', 'became', 'four', 'there', 'nowhere', 'own', 'them', 'you', 'cannot', 'but', 'first', 'of', 'though', 'often', 'three', 'itself', 'when', 'get', 'he', 'among', 'was', 'few', 'myself', 'name', 'on', 'thru', 'because', 'latterly', 'has', 'here', 'not', 'back', 'now', 'most', 'please', 'noone', 'excep

In [25]:
def remove_stop_words(x):
    x = list(map(lambda text: list(filter(lambda word: word not in ENGLISH_STOP_WORDS, text)), x))
    return x

x_train = remove_stop_words(x_train)
x_test = remove_stop_words(x_test)

# Vectorize the dataset

An example of vectorization

In [26]:
df = pd.DataFrame(dtype=bool)
for i, text in enumerate(tqdm(x_train[:5])):
    for word in text:
        df.loc[i, word] = True
df.fillna(False, inplace=True)

  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-26-c6cb43738d2c>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)


In [27]:
df

,scripture,quote,id,like,read,just,sure,?,good,need,...,driving,nuts,!,thank,advice,dont,blanket,generalizations,false,unhelpful
0,True,True,True,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,True,True


In [28]:
top_words = {}
for text in x_train:
    for word in text:
            if word in top_words:
                top_words[word] += 1
            else:
                top_words[word] = 1

top_words_ordered = sorted(top_words.items(), key=lambda kv: kv[1])
print(top_words_ordered)

[('iwreck', 1), ('2lbs', 1), ('pussycat', 1), ('seventh', 1), ('yepand', 1), ('notoriety', 1), ('usamuelzeign', 1), ('vase', 1), ('resetting', 1), ('carpenter', 1), ('livable', 1), ('silicon', 1), ('hooooold', 1), ('rapey', 1), ('bernabeu', 1), ('pewdiepies', 1), ('likedislike', 1), ('apparenlty', 1), ('incriminating', 1), ('paralegal', 1), ('birdy', 1), ('mtl', 1), ('blacklock', 1), ('mamzer', 1), ('stardawg', 1), ('ts', 1), ('poignant', 1), ('conflicts', 1), ('matt', 1), ('berry', 1), ('tldrs', 1), ('covina', 1), ('cutlery', 1), ('responsebility', 1), ('natalist', 1), ('perversion', 1), ('ift', 1), ('menopause', 1), ('515', 1), ('lapses', 1), ('rdrama', 1), ('screencaps', 1), ('rhinestones', 1), ('expresses', 1), ('redefine', 1), ('decimate', 1), ('mollycoddle', 1), ('homegirl', 1), ('laguna', 1), ('jackin', 1), ('taxpayers', 1), ('abbey', 1), ('belcalis', 1), ('61045', 1), ('whoomp', 1), ('consensually', 1), ('rosé', 1), ('managehopefully', 1), ('pooper', 1), ('attracting', 1), ('ap

In [29]:
print(list(map(lambda x: x[1], top_words_ordered)).index(1))
print(list(map(lambda x: x[1], top_words_ordered)).index(2))
print(list(map(lambda x: x[1], top_words_ordered)).index(3))
print(list(map(lambda x: x[1], top_words_ordered)).index(4))
print(list(map(lambda x: x[1], top_words_ordered)).index(5))

0
314
3269
11826
14338


In [30]:
word_list = list(map(lambda x: x[0], top_words_ordered[-1000:]))
print(len(word_list))

1000


Create an index for every unique word

In [31]:
tensor_index = {}
for i, word in enumerate(tqdm(word_list)):
    tensor_index[word] = i

  0%|          | 0/1000 [00:00<?, ?it/s]

In [32]:
print(len(tensor_index))

1000


Check synonyms

In [33]:
nltk.download('wordnet')  # Download WordNet if not already installed

def get_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name().lower())
    return synonyms - {word}

word = "happy"
print(get_synonyms(word))

[nltk_data] Downloading package wordnet to /root/nltk_data...


{'glad', 'felicitous', 'well-chosen'}


Create a tensor

In [34]:
def create_x_tensor(x, syn=False):
    x_tensor = torch.zeros(len(x), len(tensor_index), dtype=torch.float16)
    for i, text in enumerate(tqdm(x)):
        for word in text:
            if word in tensor_index:
                x_tensor[i, tensor_index[word]] += 1
                if syn:
                    for syn_word in get_synonyms(word):
                        if syn_word in tensor_index:
                            x_tensor[i, tensor_index[syn_word]] += 1
    return x_tensor

In [35]:
# x_train
# last_index = 0
# for i in range(0, len(x_train), step):
#     last_index = i
#     torch.save(create_x_tensor(x_train[i:i+step]), BASE_PATH + f'tensors/x_train_tensor{i//step}.pt')
# torch.save(create_x_tensor(x_train[last_index:]), BASE_PATH + f'tensors/x_train_tensor{last_index//step + 1}.pt')

In [36]:
torch.save(create_x_tensor(x_train, syn=True), BASE_PATH + 'tensors/x_train_tensor.pt')
torch.save(create_x_tensor(x_test), BASE_PATH + 'tensors/x_test_tensor.pt')

  0%|          | 0/190000 [00:00<?, ?it/s]

  0%|          | 0/10562 [00:00<?, ?it/s]

Convert labels to tensor

In [37]:
y_train.describe()

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
count,190000.00000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000
mean,0.15770,0.069826,0.096211,0.077153,0.333437,0.063179,0.080874,0.021284,0.127579,0.118505
std,0.36446,0.254855,0.294881,0.266834,0.471442,0.243285,0.272642,0.144331,0.333621,0.323206
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
def create_y_tensor(y):
    y_tensor = torch.tensor(y.values, dtype=torch.float16)
    return y_tensor

In [39]:
# y_train
# last_index = 0
# for i in range(0, len(y_train), step):
#     last_index = i
#     torch.save(create_y_tensor(y_train[i:i+step]), BASE_PATH + f'tensors/y_train_tensor{i//step}.pt')
# torch.save(create_y_tensor(y_train[last_index:]), BASE_PATH + f'tensors/y_train_tensor{last_index//step + 1}.pt')

In [40]:
torch.save(create_y_tensor(y_train), BASE_PATH + 'tensors/y_train_tensor.pt')
torch.save(create_y_tensor(y_test), BASE_PATH + 'tensors/y_test_tensor.pt')